# Goal: train a MNIST digit classifier CNN using only Numpy

In [49]:
import numpy as np
import matplotlib.pyplot as plt

### Get data

In [50]:
import tensorflow as tf # tensorflow is only for downloading the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

val_samples = 1000
(x_train, x_val), (y_train, y_val) = (x_train[:-1000], x_train[-1000:]), (y_train[:-1000], y_train[-1000:])

### Define model

In [66]:
model = dict()


categories = 10


# convolutional parameters 

filter_sizes = [[3,3,1], [3,3,32]]
filter_number = [32,64]
step = [2,2]
image_size = (28,28,1)

# dense parameters

number_of_dense_layers = 2
output_nodes = [128,10]
activations = ['relu', 'softmax']


current_input_size = image_size

for cnn_layer in range(len(filter_number)):
    
    d0 = filter_sizes[cnn_layer][0]
    d1 = filter_sizes[cnn_layer][1]
    d2 = filter_sizes[cnn_layer][2]
    d3 = filter_number[cnn_layer]
    
    model['F'+str(cnn_layer+1)] = np.random.randn(d0, d1, d2, d3) / np.sqrt(d0*d1*d2) # filter 1
    
    # calculate output shape
    steps_i = (current_input_size[0]+1-2*(filter_sizes[cnn_layer][0]//2))//step[cnn_layer]
    steps_j = (current_input_size[1]+1-2*(filter_sizes[cnn_layer][1]//2))//step[cnn_layer]
    print('Conv Layer {}:'.format(cnn_layer))
    print('  -  Input shape:{}'.format(current_input_size))
    current_input_size = (steps_i,steps_j,filter_number[cnn_layer])
    print('  -  Output shape:{}'.format(current_input_size))
    
    #Bias
    model['b_conv_'+str(cnn_layer+1)] = np.random.randn(current_input_size[0], current_input_size[1],
                                                   current_input_size[2]) / np.sqrt(current_input_size[2])
    
dense_input_size = np.prod(current_input_size)

for dense_layer in range(number_of_dense_layers):
    
    model['D'+str(dense_layer+1)] = np.random.randn(output_nodes[dense_layer], dense_input_size) / np.sqrt(dense_input_size)# dense layer weights
    model['b_dense_'+str(dense_layer+1)] = np.random.randn(output_nodes[dense_layer]) / np.sqrt(output_nodes[dense_layer])# dense layer weights
    
    print('Dense Layer {}:'.format(dense_layer))
    print('  -  Input shape:{}'.format(dense_input_size))
    print('  -  Output shape:{}'.format(output_nodes[dense_layer]))
    
    dense_input_size = output_nodes[dense_layer]

Conv Layer 0:
  -  Input shape:(28, 28, 1)
  -  Output shape:(13, 13, 32)
Conv Layer 1:
  -  Input shape:(13, 13, 32)
  -  Output shape:(6, 6, 64)
Dense Layer 0:
  -  Input shape:2304
  -  Output shape:128
Dense Layer 1:
  -  Input shape:128
  -  Output shape:10


### Define forward propagation

In [54]:
def softmax(y):
    return np.exp(y)/np.sum(np.exp(y))

# Convolution
def conv_forward(x,layer):
    global model, filter_number, filter_sizes, step
    
    x = x.reshape((x.shape[0],x.shape[1],filter_sizes[layer][2],1))

    x = np.tile(x,filter_number[layer])

    steps_i = (x.shape[0]+1-2*(filter_sizes[layer][0]//2))//step[layer]
    steps_j = (x.shape[1]+1-2*(filter_sizes[layer][1]//2))//step[layer]

    y = np.zeros((steps_i,steps_j,filter_number[layer]))

    
    index_i = 0
    
    for i in range(filter_sizes[layer][0]//2, x.shape[0]- filter_sizes[layer][0]//2, step[layer]):
        index_j = 0
        for j in range(filter_sizes[layer][1]//2, x.shape[1]- filter_sizes[layer][1]//2, step[layer]):
            local_x = x[i-filter_sizes[layer][0]//2: i+filter_sizes[layer][0]//2+1,j-filter_sizes[layer][1]//2:
                        j+filter_sizes[layer][0]//2+1]
            y[index_i][index_j] = np.sum(np.multiply(local_x, model['F'+str(layer+1)]).reshape(-1,filter_number[layer]), axis = 0)

            index_j+= 1
        index_i+= 1
    # Add bias
    y += model['b_conv_'+str(layer+1)]
    # ReLU
    y[y<0] = 0
    
    return y


# Dense
def dense_forward(x, dense_layer):
    global model, activations
    
    h = np.dot(model['D'+str(dense_layer+1)], x) + model['b_dense_'+str(dense_layer+1)]
    
    # activation
    
    if activations[dense_layer] == 'relu':
        y = h
        y[y<0] = 0.
    
    if activations[dense_layer] == 'softmax':
        y = softmax(h)
    
    return y,h


### Backprop now

In [56]:
def categorical_ce_grad(h, y_true):
    y_onehot = np.zeros_like(h)
    y_onehot[y_true] = 1
    grad = softmax(h)-y_onehot
    return grad

def backprop_dense(dy, h, dense_layer):
    global model

    # bias update
    db = dy

    # Weights update
    dw = np.outer(dy, h)
    
    dh = np.dot(dy, model['D'+str(dense_layer + 1)])

    return dw, db, dh

def conv_backprop(x,dh,layer):
    global model, filter_number, filter_sizes, step
    
    x = x.reshape((x.shape[0],x.shape[1],filter_sizes[layer][2],1))
    dx = np.zeros_like(x)
    x = np.tile(x,filter_number[layer])

    steps_i = (x.shape[0]+1-2*(filter_sizes[layer][0]//2))//step[layer]
    steps_j = (x.shape[1]+1-2*(filter_sizes[layer][1]//2))//step[layer]

    y = np.zeros((steps_i,steps_j,filter_number[layer]))
    
    index_i = 0
    
    #Weights update
    dw = np.zeros_like(model['F'+str(layer+1)])
    db = dh
    
    for i in range(filter_sizes[layer][0]//2, x.shape[0]- filter_sizes[layer][0]//2, step[layer]):
        index_j = 0
        for j in range(filter_sizes[layer][1]//2, x.shape[1]- filter_sizes[layer][1]//2, step[layer]):
            local_x = x[i-filter_sizes[layer][0]//2: i+filter_sizes[layer][0]//2+1,j-filter_sizes[layer][1]//2:
                        j+filter_sizes[layer][0]//2+1]
            local_dx = np.sum(np.multiply(model['F'+str(layer+1)], dh[index_i][index_j]), axis=-1)
            dx[i-filter_sizes[layer][0]//2: i+filter_sizes[layer][0]//2+1,j-filter_sizes[layer][1]//2:
                        j+filter_sizes[layer][0]//2+1,:,0] += local_dx
            dw+=np.multiply(local_x, dh[index_i][index_j])
            index_j+= 1
        index_i+= 1
    
    dx = dx.reshape(dx.shape[:-1])
    
    return dw,db,dx

import copy

#initial = copy.deepcopy(model)

def eval_model(x, y_true):
    
    h1 = conv_forward(x, 0)
    h2 = conv_forward(h1, 1)
    h2f = h2.flatten()
    y3, h3 = dense_forward(h2f, 0)
    y, h = dense_forward(y3, 1) 
    
    return y, np.argmax(y) == y_true

# Train model

In [67]:
LR = 0.0005
epochs = 10
lr = 0.0001
steps = 0

for epoch in range(epochs):


    for image_id in range(len(x_train)):
        
        if steps % 2000 == 0:
                # test accuracy
                samples = 100
                s = 0
                for e in range(samples):
                    image_id = np.random.randint(len(x_train))
                    x = x_train[image_id]
                    y_true = y_train[image_id]
                    y, good = eval_model(x, y_true)
                    if good:
                        s += 1
                print("Accuracy on training data is {}%".format(100.*s/float(samples)))
                accuracy = s/float(samples)
        
        
        if steps % 200 == 0:
                # test accuracy
                samples = 100
                s = 0
                for e in range(samples):
                    image_id = np.random.randint(len(x_val))
                    x = x_val[image_id]
                    y_true = y_val[image_id]
                    y, good = eval_model(x, y_true)
                    if good:
                        s += 1
                print("Validation Accuracy is {}%".format(100.*s/float(samples)))
                accuracy = s/float(samples)
                #lr = (1 - accuracy) * LR
            
        steps += 1
        x = x_train[image_id]
        y_true = y_train[image_id]

        # FORWARD

        h1 = conv_forward(x, 0)
        h2 = conv_forward(h1, 1)
        h2f = h2.flatten()
        y3, h3 = dense_forward(h2f, 0)
        y, h = dense_forward(y3, 1) 

        # BACKPROP

        ## Loss gradient
        grad = categorical_ce_grad(h, y_true)
        dy = -grad

        ## Dense Backprop

        dw3, db3, dh3 = backprop_dense(dy, h3, 1)
        dh3[h3<=0.] = 0. ## relu

        model['D2'] += lr * dw3
        model['b_dense_2'] += lr * db3
        
        dw2, db2, dh2 = backprop_dense(dh3, h2f, 0)
        dh2 = dh2.reshape(h2.shape)
        dh2[h2<=0] = 0 ## relu

        model['D1'] += lr * dw2
        model['b_dense_1'] += lr * db2

        ## 2nd conv layer Backprop
        dw1, db1, dh1 = conv_backprop(h1, dh2, 1)
        dh1[h1<=0] = 0 ## relu

        model['F2'] += lr * dw1
        model['b_conv_2'] += lr * db1

        ## 1st conv layer Backprop
        dw0, db0, dx0 = conv_backprop(x.astype('float64'), dh1, 0)

        model['F1'] += lr * dw0
        model['b_conv_1'] += lr * db0


Accuracy on training data is 7.0%
Validation Accuracy is 9.0%
Validation Accuracy is 59.0%
Validation Accuracy is 69.0%
Validation Accuracy is 82.0%
Validation Accuracy is 89.0%
Validation Accuracy is 80.0%
Validation Accuracy is 77.0%
Validation Accuracy is 80.0%
Validation Accuracy is 92.0%
Validation Accuracy is 84.0%
Accuracy on training data is 78.0%
Validation Accuracy is 88.0%
Validation Accuracy is 80.0%
Validation Accuracy is 89.0%
Validation Accuracy is 92.0%
Validation Accuracy is 84.0%
Validation Accuracy is 90.0%
Validation Accuracy is 93.0%
Validation Accuracy is 92.0%
Validation Accuracy is 89.0%
Validation Accuracy is 93.0%
Accuracy on training data is 89.0%
Validation Accuracy is 92.0%
Validation Accuracy is 93.0%
Validation Accuracy is 94.0%
Validation Accuracy is 94.0%
Validation Accuracy is 94.0%
Validation Accuracy is 92.0%
Validation Accuracy is 92.0%
Validation Accuracy is 94.0%
Validation Accuracy is 90.0%
Validation Accuracy is 85.0%
Accuracy on training data i

Validation Accuracy is 100.0%
Validation Accuracy is 97.0%
Validation Accuracy is 98.0%
Validation Accuracy is 96.0%
Validation Accuracy is 98.0%
Validation Accuracy is 96.0%
Validation Accuracy is 98.0%
Validation Accuracy is 94.0%
Accuracy on training data is 97.0%
Validation Accuracy is 94.0%
Validation Accuracy is 99.0%
Validation Accuracy is 96.0%
Validation Accuracy is 98.0%
Validation Accuracy is 98.0%
Validation Accuracy is 98.0%
Validation Accuracy is 98.0%
Validation Accuracy is 97.0%
Validation Accuracy is 99.0%
Validation Accuracy is 99.0%
Accuracy on training data is 96.0%
Validation Accuracy is 98.0%
Validation Accuracy is 97.0%
Validation Accuracy is 98.0%
Validation Accuracy is 97.0%
Validation Accuracy is 97.0%
Validation Accuracy is 95.0%
Validation Accuracy is 97.0%
Validation Accuracy is 93.0%
Validation Accuracy is 99.0%
Validation Accuracy is 98.0%
Accuracy on training data is 98.0%
Validation Accuracy is 93.0%
Validation Accuracy is 98.0%
Validation Accuracy is 9

KeyboardInterrupt: 

# Test on the test set

In [68]:
samples = 10000
s = 0
for e in range(samples):
    image_id = e
    x = x_test[image_id]
    y_true = y_test[image_id]
    y, good = eval_model(x, y_true)
    if good:
        s += 1
print("Accuracy is {}%".format(100.*s/float(samples)))

Accuracy is 97.03%


### Save model

In [70]:
import pickle
with open('cnn_model.pickle', 'wb') as f:
    pickle.dump(model, f)
#with open('cnn_model.pickle', 'rb') as f:
#    m = pickle.load(f)